## Multioutput model

Now, we need a model to detect the type of hate

In [1]:
%load_ext autoreload
%autoreload 2
import json

with open("../data/train.json") as f:
    train_articles = json.load(f)

with open("../data/test.json") as f:
    test_articles = json.load(f)

Let's take just the comments that are HATEFUL

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd

train_comments = [c for article in train_articles for c in article["comments"] if c["is_hateful"]]
test_comments = [c for article in test_articles for c in article["comments"] if c["is_hateful"]]

train_df = pd.DataFrame(train_comments)
test_df = pd.DataFrame(test_comments)

train_df, dev_df = train_test_split(train_df, test_size=0.2)

print(f"We have {len(train_df)} hateful comments in train")
print(f"We have {len(dev_df)} hateful comments in dev")
print(f"We have {len(test_df)} hateful comments in test")

We have 5140 hateful comments in train
We have 1285 hateful comments in dev
We have 1676 hateful comments in test


In [3]:
categories = [
    "calls",
    "WOMEN",
    "LGBTI",
    "RACISM",
    "CLASS",
    "POLITICS",
    "DISABLED",
    "APPEARANCE",
    "CRIMINAL",
]

train_df[categories].mean() - test_df[categories].mean()

calls         0.036945
WOMEN        -0.047260
LGBTI        -0.021873
RACISM        0.040700
CLASS        -0.017619
POLITICS     -0.021994
DISABLED     -0.010198
APPEARANCE   -0.014577
CRIMINAL      0.077541
dtype: float64

It is slightly unbalanced!

In [4]:
from hatedetection.preprocessing import preprocess_tweet

train_df["text"] = train_df["text"].apply(preprocess_tweet)
dev_df["text"] = dev_df["text"].apply(preprocess_tweet)
test_df["text"] = test_df["text"].apply(preprocess_tweet)


## Binary Cross Entropy Loss

Si tenemos nuestras categorías en $C$, queremos hacer un "multi-tasking" usando una loss que sea 

$$
J(y, \hat{y}) = \frac{1}{|C|}\sum\limits_{c \in C} J_c(y, \hat{y})
$$

O sea, para cada instancia, la función de pérdida va a ser el promedio de las pérdidas para `MUJER`, `RACISMO`, etc...

In [5]:
import torch
from torch.nn import BCEWithLogitsLoss
from torch.nn.functional import binary_cross_entropy_with_logits


"""
Supongamos que tenemos un batch de 32 
Por cada uno
"""

logits = torch.randn(32, 8)
labels = torch.Tensor([[1, 1, 1, 1, 0, 0, 0, 0] for _ in range(32)])


loss_fct = BCEWithLogitsLoss()
loss_fct(logits, labels)


tensor(0.7450)

In [6]:
logits = torch.Tensor([[-10, -9, -10]])
target = torch.zeros(1, 3)

loss_fct(
    logits,
    target,
)

tensor(7.1403e-05)

¿Está haciendo lo esperado esto? Veamos...

Cross entropy es 

$- [y \log \hat{y} + (1-y) \log (1-\hat{y}) ]$

In [7]:
from torch.nn.functional import sigmoid

pred = sigmoid(logits)

losses = -(target * torch.log(pred) + (1 - target) * torch.log(1-pred))

losses.mean()

/home/jmperez/.local/share/virtualenvs/hatespeech-classification-n4GdOxTz/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor(7.1410e-05)

Espectacular!!! 

Qué pasa con el weight?

In [8]:
from torch.nn.functional import sigmoid

pred = sigmoid(logits)

weights = torch.Tensor([0.5, 0.1, 0.4])

losses = -(target * torch.log(pred) + (1 - target) * torch.log(1-pred))

loss_fct = BCEWithLogitsLoss(pos_weight=weights)

(losses * weights).sum(), loss_fct(logits, target)

(tensor(5.3217e-05), tensor(7.1526e-05))



Hummm...no me queda claro **CHEQUEAR ESTO**

## Clasificación

Usamos nuestro modelo `hatedetection.BertForSequenceMultiClassification`. Es una leve modificación del clasificador de `transformers`

In [9]:
from transformers import AutoTokenizer
from hatedetection import BertForSequenceMultiClassification

model_name = 'dccuchile/bert-base-spanish-wwm-cased'

device = "cuda" if torch.cuda.is_available() else "cpu"

id2label = {0: 'Not hateful', 1: 'Hateful'}
label2id = {v:k for k,v in id2label.items()}

model = BertForSequenceMultiClassification.from_pretrained(model_name, return_dict=True, num_labels=len(categories))

model.config.id2label = id2label
model.config.label2id = label2id

model = model.to(device)
model.train();



tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceMultiClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceMultiClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceMultiClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceMultiClassification were not initialized from the model checkpoint at dccuchile/bert-base

In [10]:
from datasets import Dataset, Value, ClassLabel, Features

#examples = pd.concat([train_df, dev_df])

features = Features({
    'text': Value('string'),
})

for cat in categories:
    features[cat] = ClassLabel(num_classes=2, names=["NO", "YES"])

columns = ["text"] + categories
train_dataset = Dataset.from_pandas(train_df[columns], features=features)
dev_dataset = Dataset.from_pandas(dev_df[columns], features=features)
test_dataset = Dataset.from_pandas(test_df[columns], features=features)


In [11]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

batch_size = 32
eval_batch_size = 16

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
dev_dataset = dev_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)



In [12]:
def format_dataset(dataset):
    def sarasa(examples):
        return {'labels': torch.Tensor([examples[cat] for cat in categories])}
    dataset = dataset.map(sarasa)
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)

Esta API de mierda vive cambiando todo el tiempo

In [13]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    """
    Compute metrics for Trainer
    """
   
    
    
    labels = pred.label_ids
    preds = torch.sigmoid(torch.Tensor(pred.predictions)).round()

    ret = {
    }
    """
    Calculo F1 por cada posición. Asumo que cada categoría está alineada correctamente en la i-ésima posición
    """
    f1s = []
    for i, cat in enumerate(categories):
        cat_labels, cat_preds = labels[:, i], preds[:, i]
        precision, recall, f1, _ = precision_recall_fscore_support(cat_labels, cat_preds, average='macro')
        
        f1s.append(f1)
        
        ret[cat+" F1"] = f1
        
    ret["Mean F1"] = torch.Tensor(f1s).mean()
    return ret

In [14]:
from transformers import Trainer, TrainingArguments
epochs = 5

total_steps = (epochs * len(train_dataset)) // batch_size
warmup_steps = total_steps // 10

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    warmup_steps=warmup_steps,
    evaluation_strategy="epoch",
    do_eval=False,
    weight_decay=0.01,
    logging_dir='./logs',
)

results = []

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Calls f1,Women f1,Lgbti f1,Racism f1,Class f1,Politics f1,Disabled f1,Appearance f1,Criminal f1,Mean f1,Runtime,Samples Per Second
1,No log,0.247293,0.824837,0.709522,0.669156,0.894917,0.650340,0.788101,0.569254,0.872706,0.836701,0.757282,5.275600,243.572000
2,No log,0.193201,0.823014,0.813078,0.754359,0.910956,0.797217,0.844851,0.794621,0.903271,0.873970,0.835037,5.355500,239.939000
3,No log,0.183866,0.848037,0.821601,0.798580,0.920349,0.804524,0.847298,0.864021,0.904878,0.862286,0.852397,5.358500,239.807000
4,0.236600,0.183888,0.853212,0.820883,0.802736,0.922187,0.840342,0.840796,0.849010,0.909721,0.867442,0.856259,5.252500,244.646000
5,0.236600,0.182855,0.857016,0.821241,0.812675,0.925905,0.817732,0.847217,0.864239,0.912296,0.867442,0.858418,5.336600,240.789000


TrainOutput(global_step=805, training_loss=0.178641398649038, metrics={'train_runtime': 506.1865, 'train_samples_per_second': 1.59, 'total_flos': 2168329333017600.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 55269, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 593768, 'train_mem_gpu_alloc_delta': 1381010944, 'train_mem_cpu_peaked_delta': 191444750, 'train_mem_gpu_peaked_delta': 3558643712})

In [62]:
def predict(sentence, context=None):
    sentence = preprocess_tweet(sentence)
    idx = torch.LongTensor(tokenizer.encode(sentence)).view(1, -1).to(device)
    
    output = model(idx)

    output = output.logits.detach().cpu().numpy()[0]

    ret = [cat for cat, out in list(zip(categories, output > 0)) if out]

    return ret

In [63]:
predict("Negro chorro de mierda")

['RACISM']

In [64]:
predict("Quiere bijaaa")

['WOMEN']

In [65]:
predict("Le falta verga")

['LGBTI']

In [66]:
predict("Trolazo")

['LGBTI']

In [67]:
predict("Te llamás Raúl querido")

['LGBTI']

In [68]:
predict("No seas trolo viejo")

['LGBTI', 'APPEARANCE']

In [69]:
predict("A esta trola hay que agarrarla de a varios para que sepa lo que es bueno")

['WOMEN']

Acá no encuentra el llamado a la acción...ahora, si lo cambiamos ligeramente...

In [70]:
predict("A esta negra de mierda hay que agarrarla de a varios para que sepa lo que es bueno")

['calls', 'RACISM']

In [71]:
predict("Hay que encerrar a todos estos homosexuales de mierda")

['calls']

In [72]:
predict("Qué asco de homosexuales")

['LGBTI']

In [73]:
predict("Qué asco de homosexuales, mátenlos")

['calls', 'CRIMINAL']

Parece que correlaciona todo llamado a la acción a los criminales

In [74]:
predict("Negros chorros de mierda, cuarenta ladrillos y arreglamos esto")

['calls', 'RACISM', 'CLASS']

In [75]:
predict("negras al paredón!")

['calls', 'RACISM', 'CRIMINAL']

In [76]:
predict("No queremos rojitos acá")

['RACISM']

In [77]:
predict("No les gusta laburar eh")

['CLASS']

In [78]:
predict("Agarrá la pala!")

[]

In [79]:
predict("Agarrá la pala, vago!")

['CLASS']

In [80]:
predict("Menos ganas de laborar eh")

['CLASS']

In [81]:
predict("Viven del Estado, esta gente sinceramente no tiene remedio")

['CLASS']

In [82]:
predict("Tenés dos pelotas entre las piernas amigazo")

['LGBTI']

In [83]:
predict("jajajajaj pero ni por puta casualidad laburar, no?")

['CLASS']

In [84]:
predict("más topu imposible")

['CRIMINAL']

In [90]:
predict("hay que mandarlos a un campo de concentración a estos musulmanes")

['calls', 'RACISM']